In [ ]:
##### THIS DOESN'T WORK ANYMORE DUE TO ACCESS ISSUE EVEN AFTER USING HUGGINGFACE LOGIN USING CLI #####
# OSError: You are trying to access a gated repo.
# Make sure to request access at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 
# and pass a token having permission to this repo either by logging in with `huggingface-cli login` 
# or by passing `token=<your_token>`.

In [ ]:
# https://wandb.ai/byyoung3/ml-news/reports/Fine-Tuning-Mistral7B-on-Python-Code-With-A-Single-GPU---Vmlldzo1NTg0NzY5
# https://github.com/bdytx5/mistral7B_finetune/tree/main

# HuggingFace's Transformers - handles tokenization and model initialization
# Peft - accelerate deep learning models
# QLoRA (Quantized LoRA) - 4-bit quanitization and paged optimizers
# WandB - experiment tracking

# LoRA technique: 
# - leave the original pre-trained backbone model intact, and add on additional layers which can be more efficiently trained.
# - This allows for faster adaptability to new tasks without retraining the entire network.
# - Opens up the possibility of having an universal model that can be fine-tuned for various tasks in a more storage- and compute-efficient manner.

# Key features of QLoRA:
# - Usage of 4 bit NormalFloat datatype that is better than other 4-bit types.
# - Double Quantization further reduces memory requirements.
# - Paged Optimizers automatically transfers data between CPU and GPU to avoid OOM errors in GPU.

In [ ]:
# Setup weights and biases training metrics tracking

##### WARNING #####
# Ignore the below warning for now as it doesn't affect the functionality of wandb metric collection in any way.
# Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
##### ####### #####

import wandb
import os

wandb.login()

wandb_project = "02-finetune-mistral"
wandb_notebook_name = "02_finetune_mistral.ipynb"

if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
    os.environ["WANDB_NOTEBOOK_NAME"] = wandb_notebook_name

wandb.init(
    # set the wandb project where this run will be logged
    project=wandb_project,
    dir="../",
)

In [ ]:
import os
import shutil
from datetime import datetime
import torch
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from peft import LoraConfig
from peft import AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

def gen_batches_train():
    ds = load_dataset('json', data_files='../data/train_CRM_data.json', split='train')
    
    for sample in iter(ds):
        question = sample['messages'][1]['content']
        context = sample['messages'][0]['content']
        answer = sample['messages'][2]['content']
        
        new_text_format = f'<s>[INST] {question}\n{context} [/INST] ```python\n{answer}```</s>'
        
        tokenized_output = tokenizer(new_text_format)
        yield {'text': new_text_format}

def gen_batches_eval():
    ds = load_dataset('json', data_files='../data/eval_CRM_data.json', split='train')
    
    for sample in iter(ds):
        question = sample['messages'][1]['content']
        context = sample['messages'][0]['content']
        answer = sample['messages'][2]['content']
        
        new_text_format = f'<s>[INST] {question}\n{context} [/INST] ```python\n{answer}```</s>'
        
        tokenized_output = tokenizer(new_text_format)
        yield {'text': new_text_format}

def gen_batches_test():
    ds = load_dataset('json', data_files='../data/test_CRM_data.json', split='train')
    
    for sample in iter(ds):
        question = sample['messages'][1]['content']
        context = sample['messages'][0]['content']
        answer = sample['messages'][2]['content']
        
        new_text_format = f'<s>[INST] {question}\n{context} [/INST] ```python\n{answer}```</s>'
        
        tokenized_output = tokenizer(new_text_format)
        yield {'text': new_text_format}

def create_and_prepare_model():
    model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    
    bnb_4bit_compute_dtype = "float16"
    use_4bit = True
    bnb_4bit_quant_type = "nf4"
    use_nested_quant = False
    device_map = "auto"
    
    lora_alpha = 16
    lora_dropout = 0.1
    lora_r = 32
    
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

#     if compute_dtype == torch.float16 and use_4bit:
#         major, _ = torch.cuda.get_device_capability()
#         if major >= 8:
#             print("=" * 80)
#             print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
#             print("=" * 80)

#     # Load the entire model on the GPU 0
#     # switch to `device_map = "auto"` for multi-GPU
#     device_map = {"": 0}

    model = AutoModelForCausalLM.from_pretrained(
        model_name, 
        quantization_config=bnb_config, 
        device_map=device_map, 
        token=True,
    )
    
    # #### LLAMA STUFF 
    # # check: https://github.com/huggingface/transformers/pull/24906
    # model.config.pretraining_tp = 1 
    # # model.config.
    # #### LLAMA STUFF 
    # model.config.window = 256 

    peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM", 
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            "lm_head",
        ],
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_name, 
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token

    return model, peft_config, tokenizer

iter_str = "02"
project = "finetune"
base_model_name = "mistral"
run_name = iter_str + "-" + project + "-" + base_model_name
output_dir = "../models/" + run_name

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 50
logging_steps = 50
learning_rate = 2e-5
fp16 = False
bf16 = True
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
group_by_length = True
lr_scheduler_type = "constant"

max_seq_length = 1024
packing = False

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    evaluation_strategy="steps", 
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to='wandb',
)

model, peft_config, tokenizer = create_and_prepare_model()
model.config.use_cache = False
train_gen = Dataset.from_generator(gen_batches_train)
eval_gen = Dataset.from_generator(gen_batches_eval)
test_gen = Dataset.from_generator(gen_batches_test)
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    train_dataset=train_gen,
    eval_dataset=val_gen, 
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()